In [ ]:
# !pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose
# !pip install huggingface-hub
# !pip install --upgrade huggingface_hub
# !pip install ipywidgets
# !pip install pandas
# !pip install datasets

In [ ]:
from llama_cpp import Llama
import os
import time
!huggingface-cli login --token hf_
!huggingface-cli whoami
import time
import pandas as pd
import json
from datasets import load_dataset 
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

# Specify the new directory you want to change to
new_directory = r"Your-Path\Mistral-7B-Instruct-v0.2-GGUF"

# Change the current working directory
os.chdir(new_directory)

# Verify the change
print("Current Working Directory:", os.getcwd())

In [ ]:
llm = Llama(model_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf",
              chat_format="mistral-instruct",
              max_tokens=256,
              seed=42,
              n_gpu_layers=-1,  # Offload all layers to GPU
              main_gpu=0  # Specify the main GPU to use
              )

In [ ]:
test = load_dataset('taoofstefan/function_call_weather', split='test')
print(test)

In [ ]:
df = pd.DataFrame(test)
df.head()

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    }
]

In [ ]:
system_message = "You are a helpful assistant with access to functions. Use the provided function to answer current weather questions."

In [ ]:
# Outer loop for multiple iterations
num_iterations = 1  # Number of times the loop should run

for j in range(1, num_iterations + 1):  # Start from 1 for correct column numbering
    # Record start time for measuring loop duration
    start = time.time()

    # Loop through the DataFrame's questions
    for i in range(0, len(df)):
        # Retrieve the current question from the DataFrame
        question = df["Question"][i]
        
        # Define the message structure for the LLM inference
        message = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ]
        
        # Run the LLM inference
        result = llm.create_chat_completion(
            messages=message,
            tools=tools[0]['function'],
            tool_choice="auto",    
        )
        
        # Store the assistant's response content, cleaned
        assistant_response = result["choices"][0]["message"]["content"]
            
        # Store in respective columns
        df.at[i, f"Result_{j}"] = assistant_response

        print(result)
    
    # Record the end time to measure loop duration
    end = time.time()

    # Calculate the total duration for this `j` iteration
    duration = end - start

In [ ]:
print("Time LLM worked:", str(duration))

In [ ]:
df_clean = df

In [ ]:
# Function to clean text if it doesn't start with a '{'
def clean_if_not_json(text):
    if isinstance(text, str):
        if text.startswith("{"):
            # No cleaning required
            return text
        else:
            # Strip all commas
            return text.replace(",", "").strip()
    return text

# List of columns to apply the cleaning function
# columns_to_clean = ["Result_1", "Result_2", "Result_3"]
columns_to_clean = ["Result_1"]

# Apply the cleaning function to the specified columns
for column in columns_to_clean:
    df[column] = df[column].apply(clean_if_not_json)

# Display the cleaned DataFrame
df.head()

In [ ]:
# Save to CSV with a specified delimiter
df.to_csv(r"Your-Path\result_mistral_chat_completion_format.csv", sep=";", index=False)